In [1]:
import motrainer
import numpy as np
import xarray as xr

In [2]:
import dask
dask.config.set(scheduler='synchronous')

## Load data

In [3]:
ds = xr.open_zarr("../../example/example1_data.zarr/")
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    longitude  (space) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
Dimensions without coordinates: space
Data variables:
    BIOMA1     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    BIOMA2     (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    TG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG1        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG2        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    WG3        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    curv       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    sig        (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
    slop       (space, time) float64 dask.array<chunksize=(3, 8506), meta=np.ndarray>
Attributes:
    license:  data license
    source:   data source

## Split per gridcell

In [4]:
ds = ds.compute()
ds

<xarray.Dataset>
Dimensions:    (space: 5, time: 8506)
Coordinates:
    latitude   (space) float64 56.12 46.12 53.38 49.38 44.38
    longitude  (space) float64 11.38 6.625 6.125 12.38 0.625
  * time       (time) datetime64[ns] 2007-01-02 ... 2020-01-01T01:00:00
Dimensions without coordinates: space
Data variables:
    BIOMA1     (space, time) float64 0.07079 0.05532 0.04846 ... nan nan nan
    BIOMA2     (space, time) float64 0.04366 0.0462 0.03821 ... nan nan nan
    TG1        (space, time) float64 280.0 270.4 285.5 277.4 ... nan nan nan nan
    TG2        (space, time) float64 274.8 278.4 280.6 283.7 ... nan nan nan nan
    TG3        (space, time) float64 280.9 279.7 278.0 278.0 ... nan nan nan nan
    WG1        (space, time) float64 0.3249 0.2798 0.2773 0.2867 ... nan nan nan
    WG2        (space, time) float64 0.3408 0.2902 0.3373 0.2709 ... nan nan nan
    WG3        (space, time) float64 0.3123 0.2916 0.2891 0.3538 ... nan nan nan
    curv       (space, time) float64 -0.001396 -0.001464 ... -0.0006173
    sig        (space, time) float64 -9.49 -8.494 -9.069 ... -8.071 -8.237
    slop       (space, time) float64 -0.1208 -0.1178 -0.121 ... -0.1144 -0.1191
Attributes:
    license:  data license
    source:   data source

In [5]:
motrainer.is_splitable(ds)

True

In [6]:
bags = motrainer.dataset_split(ds, "space")
bags

dask.bag<from_sequence, npartitions=5>

In [12]:
identifier = {"space": np.array([0, 1, 1, 0, 1])}
bags = motrainer.dataset_split(ds, identifier)

ds_splitted = bags.compute()
print(ds_splitted[0].dims['space'])
print(ds_splitted[1].dims['space'])

2
3


In [17]:
id_time = np.zeros(8506)
id_time[2000:]=1
identifier = {"space": np.array([0, 1, 1, 0, 1]), "time": id_time}
bags = motrainer.dataset_split(ds, identifier)

print(bags)

dask.bag<from_sequence, npartitions=4>


In [16]:
for parts in ds_splitted:
    print(parts.dims)

Frozen({'samples': 4000})
Frozen({'samples': 13012})
Frozen({'samples': 6000})
Frozen({'samples': 19518})


## Train Test Split

In [ ]:
def to_dataframe(ds):
    return ds.to_dask_dataframe()

def chunk(ds, chunks):
    return ds.chunk(chunks)

In [ ]:
# Train test split
train_test_bags = bags.map(
    motrainer.train_test_split, split={"time": np.datetime64("2016-01-01")}
)

# # Or split by mask
# mask = ds["time"]<np.datetime64("2016-01-01")
# train_test_bags = bags.map(
#     motrainer.train_test_split, mask={"time": np.datetime64("2016-01-01")}
# )

In [ ]:
train_bags = train_test_bags.pluck(0).map(chunk, {"space": 500}).map(to_dataframe)
test_bags = train_test_bags.pluck(1).map(chunk, {"space": 500}).map(to_dataframe)

## Setup Training

In [ ]:
# Setup grid search
# use the estimator definition and pipeline objects in sklearn
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold

from dask_ml.preprocessing import MinMaxScaler
from dask_ml.model_selection import GridSearchCV

regSVR = make_pipeline(MinMaxScaler(), SVR())
kernel = ["poly", "rbf", "sigmoid"]
C = [1, 0.1]
gamma = ["scale"]
grid = dict(svr__kernel=kernel, svr__C=C, svr__gamma=gamma)
cv = RepeatedKFold(n_splits=4, n_repeats=2, random_state=1)
grid_search = GridSearchCV(
    estimator=regSVR,
    param_grid=grid,
    cv=cv,
    scoring=["r2", "neg_mean_squared_error"],
    refit="r2",
)

## Model Optimization

In [ ]:
def optimize(df, grid_search, input_list, output_list):
    """Customized Optimization Function
    """
    df = df.dropna()
    grid_result = grid_search.fit(df[input_list], df[output_list])
    return grid_result


input_list = ["BIOMA1", "BIOMA1", "TG1", "TG2", "TG3"]
output_list = ["slop"]
optimazed_estimators = train_bags.map(
    optimize, grid_search=grid_search, input_list=input_list, output_list=output_list
)

In [ ]:
optimazed_estimators_realized = optimazed_estimators.compute()

## Save model

In [ ]:
# To be replaced by "modelstore"
import pickle

for model, id in zip(optimazed_estimators_realized, range(len(optimazed_estimators_realized))):
    name_model = f"model{id}.pickle"
    with open(name_model, "wb") as f:
        pickle.dump(model, f)

## Model performance evaluation

In [ ]:
# Load the models back
list_model = []
for id in range(5): 
    with open(f"model{id}.pickle", "rb") as f:
        list_model.append(pickle.load(f))
list_model

In [ ]:
from sklearn.metrics import mean_squared_error , r2_score,  mean_absolute_error

# This for need to be coverted to a user defined 
list_metrics = []
for model, test_data in zip(list_model, test_bags.compute()):
    test_data = test_data.dropna()
    X_test = test_data[input_list]
    Y_test = test_data[output_list]
    Y_eval = model.predict(X_test)

    metrics = {"MSE_SVR": mean_squared_error(Y_test,Y_eval),
               "MAE_SVR": mean_absolute_error(Y_test,Y_eval),
               "R_2":r2_score(Y_test,Y_eval)}
    list_metrics.append(metrics)